## Building a Targeted Dataset for Ticket Price Prediction  

To develop a predictive model for ticket pricing, I engineered a dataset by integrating features that significantly influence price fluctuations. Key variables included event popularity metrics, supporting artist prominence scores, proximity to the event date (days until event), and additional determinants of consumer demand. The dataset was constructed by requesting data from APIs, extracting relevant features, and merging them using shared identifiers to ensure consistency and usability.

#### Data Pipeline and Sources

The data pipeline utilized multiple APIs to extract and preprocess relevant features:

- **Spotify API**: Retrieved structured data on artist popularity indices, years active, and associated genres for encoding.

- **Reddit API**: Scraped real-time mentions of artists, applying filtering logic to capture relevant mentions from the current day to gauge social buzz.

- **Ticketmaster API**: Collected granular details on ticket listings, event timelines, and venue attributes for feature engineering.

In [1]:
import os
import base64
import requests
import lyricsgenius as lg
import pandas as pd
import time 
from googleapiclient.discovery import build 
from difflib import get_close_matches
from datetime import datetime, timezone
from dotenv import load_dotenv
import re
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# Load the .env file
load_dotenv("Data_api.env")

# Access credentials
SPOTIFY_ID = os.getenv("SPOTIFY_ID")
SPOTIFY_SECRET = os.getenv("SPOTIFY_SECRET")
GENIUS_ACCESS_TOKEN = os.getenv("GENIUS_ACCESS_TOKEN")
TICKETMASTER_KEY = os.getenv("MASTER_KEY")

## Extracting LA Venue Identifiers

**This function fetches all music-related venues from the Ticketmaster API. It uses pagination to retrieve data in batches, processes the response to extract relevant details such as venue name, address, and classification, and returns a DataFrame of venues.**

In [3]:
def get_all_music_venues():
    
    url = "https://app.ticketmaster.com/discovery/v2/venues.json"
    params = {
        "apikey": TICKETMASTER_KEY,
        "classificationName": "music",  # Filter for music-related venues
        "countryCode": "US",
        "size": 10,  # Max size per request
        "page": 0     # Start with the first page
    }

    all_venues = []
    while True:
        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"Error fetching venues: {response.text}")
            break

        data = response.json()
        if "_embedded" not in data or "venues" not in data["_embedded"]:
            print("No more venues found.")
            break

        for venue in data["_embedded"]["venues"]:
            try:
                all_venues.append({
                    "venue_id": venue.get("id", "N/A"),
                    "venue_name": venue.get("name", "N/A"),
                    "address": venue.get("address", {}).get("line1", "N/A"),
                    "city": venue.get("city", {}).get("name", "N/A"),
                    "state": venue.get("state", {}).get("stateCode", "N/A"),
                    "postal_code": venue.get("postalCode", "N/A"),
                    "classification": venue.get("classifications", [{}])[0].get("segment", {}).get("name", "N/A")
                })
            except KeyError as e:
                print(f"Missing key: {e}, skipping venue.")

        # Pagination: Stop if no more pages are available
        if "page" in data and data["page"]["number"] < data["page"]["totalPages"] - 1:
            params["page"] += 1
        else:
            break

    return pd.DataFrame(all_venues)

# Fetch all music venues
all_music_venues_df = get_all_music_venues()

# Convert postal_code column to numeric, handling non-numeric values as NaN
all_music_venues_df['postal_code'] = pd.to_numeric(all_music_venues_df['postal_code'], errors='coerce')

# Filter for Los Angeles postal codes (90001 to 93552)
la_music_venues_df = all_music_venues_df[
    (all_music_venues_df['postal_code'] >= 90001) & 
    (all_music_venues_df['postal_code'] <= 93552)
]

# Display the DataFrame with Los Angeles music venues
la_music_venues_df

Error fetching venues: {"errors":[{"code":"DIS1035","detail":"API Limits Exceeded: Max paging depth exceeded. (page * size) must be less than 1,000","status":"400","_links":{"about":{"href":"/discovery/v2/errors.html#DIS1035"}}}]}


,venue_id,venue_name,address,city,state,postal_code,classification
4,KovZ917ACh0,SoFi Stadium,1001 S. Stadium Dr,Inglewood,CA,90301,N/A
21,KovZpZAEkn6A,Kia Forum,3900 W Manchester Blvd.,Inglewood,CA,90305,N/A
35,KovZpakTme,Honda Center,2695 E. Katella,Anaheim,CA,92806,N/A
42,KovZpZAEeFaA,Hollywood Pantages Theatre,6233 Hollywood Blvd.,Los Angeles,CA,90028,N/A
51,KovZpZAEdntA,Crypto.com Arena,1111 S. Figueroa St,Los Angeles,CA,90015,N/A
...,...,...,...,...,...,...,...
913,KovZpZAaanlA,California Theatre of the Performing Arts,562 W. 4th St,San Bernardino,CA,92401,N/A
949,KovZ917AR1r,Calvary Community Church Westlake,5495 Via Rocas,Westlake Village,CA,91362,N/A
981,KovZpZAEvn6A,Pappy + Harriet's Pioneertown Palace,53688 Pioneertown Rd.,Pioneertown,CA,92268,N/A
983,KovZ917AGEf,The Parish at House of Blues Anaheim,"400 W Disney Way, #337",Anaheim,CA,92802,N/A


## Function to Extract Infromation using API's

This code integrates Spotify and Ticketmaster APIs to fetch, process, and analyze data related to music events, artists, and venues. It retrieves event details, artist popularity, ticket information, and additional metadata to create a structured dataset for further analysis. The final output includes enriched data with features like ticket prices, artist details, and event timelines.

### List of Functions

1. **`format_string(value)`**: Formats a string to title case; returns "N/A" if the input is invalid.
2. **`extract_ticket_limit(ticket_limit_str)`**: Extracts numerical ticket limits from a given string using regular expressions.
3. **`get_years_active(artist_name)`**: Calculates the number of years an artist has been active based on Spotify album release data.
4. **`get_total_tickets(event_id)`**: Retrieves the total number of tickets available for an event from the Ticketmaster API.
5. **`get_spotify_supporting_artist_data(supporting_artists)`**: Calculates the average Spotify popularity score for a list of supporting artists.
6. **`get_spotify_data(artist_name)`**: Fetches Spotify metadata for a given artist, including popularity, followers, genre, and years active.
7. **`get_future_concerts_for_venue(venue_id)`**: Extracts upcoming concert details for a specified venue, including artist information, ticket prices, and event timelines. Processes this data into a pandas DataFrame for analysis.

In [4]:
# Set up Spotify client
spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=SPOTIFY_ID, client_secret=SPOTIFY_SECRET))

# Utility Functions
def format_string(value):
    """
    Format a string to title case if it exists, otherwise return 'N/A'.
    """
    if isinstance(value, str):
        return value.title()
    return "N/A"

def extract_ticket_limit(ticket_limit_str):
    """
    Extract the numerical ticket limit from a string.
    """
    if isinstance(ticket_limit_str, str):
        match = re.search(r'\b\d+\b', ticket_limit_str)
        if match:
            return int(match.group())
    return None

def get_years_active(artist_name):
    """
    Calculate the number of years the artist has been active based on Spotify album release dates.
    """
    try:
        results = spotify.search(q=f"artist:{artist_name}", type="artist", limit=1)
        if not results['artists']['items']:
            return "N/A"

        artist_id = results['artists']['items'][0]['id']
        albums = spotify.artist_albums(artist_id=artist_id, album_type="album", limit=50)

        if not albums['items']:
            return "N/A"

        # Extract the earliest album release year
        release_years = []
        for album in albums['items']:
            release_date = album.get("release_date", None)
            if release_date:
                release_years.append(int(release_date[:4]))

        if release_years:
            earliest_year = min(release_years)
            current_year = datetime.now().year
            return current_year - earliest_year
        return "N/A"
    except Exception as e:
        print(f"Error calculating years active for {artist_name}: {e}")
        return "N/A"

def get_total_tickets(event_id):
    """
    Fetch the total number of tickets available for an event using Ticketmaster API.
    """
    try:
        url = f"https://app.ticketmaster.com/discovery/v2/events/{event_id}.json"
        params = {"apikey": TICKETMASTER_KEY}
        response = requests.get(url, params=params)

        if response.status_code == 200:
            event_data = response.json()
            ticket_limit = event_data.get("ticketLimit", {}).get("info", None)
            if ticket_limit:
                return extract_ticket_limit(ticket_limit)

            available_tickets = event_data.get("seatmap", {}).get("staticUrl", None)
            return available_tickets if available_tickets else "N/A"
        else:
            print(f"Error fetching total tickets for event ID {event_id}: {response.text}")
            return "N/A"
    except Exception as e:
        print(f"Error fetching total tickets for event ID {event_id}: {e}")
        return "N/A"

def get_spotify_supporting_artist_data(supporting_artists):
    """
    Fetch Spotify popularity scores for supporting artists.
    If no supporting artists, return 0.0.
    Calculate the average popularity if multiple supporting artists exist.
    """
    try:
        if not supporting_artists:
            return 0.0

        popularity_scores = []
        for artist_name in supporting_artists:
            results = spotify.search(q=f"artist:{artist_name}", type="artist", limit=1)
            if results['artists']['items']:
                artist = results['artists']['items'][0]
                popularity_scores.append(artist['popularity'])

        return round(sum(popularity_scores) / len(popularity_scores), 1) if popularity_scores else 0.0
    except Exception as e:
        print(f"Error fetching supporting artist data: {e}")
        return 0.0

def get_spotify_data(artist_name):
    """
    Fetch Spotify data for a given artist.
    """
    try:
        results = spotify.search(q=f"artist:{artist_name}", type="artist", limit=1)
        if results['artists']['items']:
            artist = results['artists']['items'][0]
            years_active = get_years_active(artist_name)
            return {
                "Spotify Artist Name": artist['name'],
                "Popularity Score": artist['popularity'],
                "Followers": artist['followers']['total'],
                "Genre": artist['genres'][0] if artist['genres'] else "N/A",
                "Years Active": years_active
            }
        else:
            return {
                "Spotify Artist Name": "N/A",
                "Popularity Score": None,
                "Followers": None,
                "Genre": "N/A",
                "Years Active": "N/A"
            }
    except Exception as e:
        print(f"Error fetching Spotify data for {artist_name}: {e}")
        return {
            "Spotify Artist Name": "N/A",
            "Popularity Score": None,
            "Followers": None,
            "Genre": "N/A",
            "Years Active": "N/A"
        }

# Main Function to Extract Data
def get_future_concerts_for_venue(venue_id):
    """
    Fetch future concert events for a specific venue using Ticketmaster API.
    Only include events where the main artist has a Spotify profile.
    """
    url = "https://app.ticketmaster.com/discovery/v2/events.json"
    params = {
        "apikey": TICKETMASTER_KEY,
        "venueId": venue_id,
        "classificationName": "music",
        "size": 200,
        "page": 0
    }

    all_events = []
    while True:
        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"Error: {response.status_code}, {response.text}")
            break

        data = response.json()

        if "_embedded" not in data or "events" not in data["_embedded"]:
            print("No more events found.")
            break

        for event in data["_embedded"]["events"]:
            try:
                # Extract artist information
                if "_embedded" in event and "attractions" in event["_embedded"]:
                    artist_names = [
                        format_string(artist["name"])
                        for artist in event["_embedded"]["attractions"]
                        if "name" in artist
                    ]
                    main_artist = artist_names[0] if artist_names else None
                    supporting_artists = artist_names[1:] if len(artist_names) > 1 else []
                    has_supporting_artists = "Yes" if supporting_artists else "No"
                else:
                    continue

                # Fetch Spotify data for the main artist
                if not main_artist:
                    continue

                spotify_data = get_spotify_data(main_artist)

                # Skip events where the main artist has no valid Spotify profile
                if (
                    spotify_data["Popularity Score"] is None
                    or spotify_data["Spotify Artist Name"] == "N/A"
                ):
                    continue

                # Fetch total tickets available for the event
                total_tickets = get_total_tickets(event["id"])
                
                
                public_onsale = event.get("sales", {}).get("public", {}).get("startDateTime", "N/A")
                presales = event.get("sales", {}).get("presales", [])
                presale_names = [presale.get("name", "N/A") for presale in presales]

                today = datetime.now()
                event_date = datetime.strptime(event["dates"]["start"]["localDate"], "%Y-%m-%d")
                public_onsale_date = (
                    datetime.strptime(public_onsale, "%Y-%m-%dT%H:%M:%SZ")
                    if public_onsale != "N/A"
                    else None
                )

                days_to_event = (event_date - today).days
                days_since_onsale = (
                    (today - public_onsale_date).days if public_onsale_date else "N/A"
                )
                
                # Extract additional event details
                min_ticket_price = event.get("priceRanges", [{}])[0].get("min", None)
                max_ticket_price = event.get("priceRanges", [{}])[0].get("max", None)

                if min_ticket_price is None and max_ticket_price is None:
                    continue

                # Parse dates
                event_date = datetime.strptime(event["dates"]["start"]["localDate"], "%Y-%m-%d")

                # Compile event details
                event_details = {
                    "Event ID": event["id"],
                    "Event Name": format_string(event["name"]),
                    "Venue Name": event.get("_embedded", {}).get("venues", [{}])[0].get("name", "N/A"),
                    "Main Artist": main_artist,
                    "Supporting Artists": ", ".join(supporting_artists) if supporting_artists else "N/A",
                    "Supporting Artist Popularity": get_spotify_supporting_artist_data(supporting_artists),
                    "Total Tickets": total_tickets,
                    "Date of Show": event["dates"]["start"]["localDate"],
                    "Min Ticket Price": min_ticket_price,
                    "Max Ticket Price": max_ticket_price,
                    "Spotify Artist Name": spotify_data["Spotify Artist Name"],
                    "Popularity Score": spotify_data["Popularity Score"],
                    "Followers": spotify_data["Followers"],
                    "Genre": spotify_data["Genre"],
                    "Years Active": spotify_data["Years Active"],
                    "Public Onsale Date": public_onsale,
                    "Days to Event": days_to_event,
                    "Days Since Onsale": days_since_onsale
                }

                all_events.append(event_details)

            except Exception as e:
                print(f"Error processing event: {e}")

        # Handle pagination
        if "page" in data and data["page"]["number"] < data["page"]["totalPages"] - 1:
            params["page"] += 1
        else:
            break

    # Create a DataFrame from all events
    df = pd.DataFrame(all_events)
    if not df.empty:
        df['Day of Week'] = pd.to_datetime(df['Date of Show']).dt.day_name()
        df['Is Weekend'] = df['Day of Week'].apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)

    return df


## List of LA Venues

**This section selects and lists the music venues located in Los Angeles from the previously fetched dataset. It extracts two columns: `venue_id` and `venue_name`, to prepare a DataFrame for further processing and analysis.**

In [5]:
# Select the LA venues with their IDs and names
la_venues = la_music_venues_df[['venue_id', 'venue_name']]
la_venues

,venue_id,venue_name
4,KovZ917ACh0,SoFi Stadium
21,KovZpZAEkn6A,Kia Forum
35,KovZpakTme,Honda Center
42,KovZpZAEeFaA,Hollywood Pantages Theatre
51,KovZpZAEdntA,Crypto.com Arena
...,...,...
913,KovZpZAaanlA,California Theatre of the Performing Arts
949,KovZ917AR1r,Calvary Community Church Westlake
981,KovZpZAEvn6A,Pappy + Harriet's Pioneertown Palace
983,KovZ917AGEf,The Parish at House of Blues Anaheim


## Aggregating Event Data Across LA Venues

Loops through the list of LA venues and fetches future music events for each venue using the `get_future_concerts_for_venue()` function. Adds metadata such as venue ID and name to the fetched event details and consolidates the data into a single DataFrame for all venues. The results include event details, artist information, and ticket availability.

**Steps**:
- Initializes an empty list to store event data for all venues.
- Iterates through the `la_venues` DataFrame to fetch events for each venue.
- Prints progress messages for each venue.
- Appends event details to a consolidated list if valid events are found.
- Combines the data from all venues into a single DataFrame for analysis.

In [6]:
# Initialize an empty list to store data for all venues
all_venues_events = []

# Loop through the top_10_venues DataFrame
for _, row in la_venues.iterrows():
    venue_id = row["venue_id"]
    venue_name = row["venue_name"]
    
    # Print the fetching message with only the venue name in bold and blue
    print(f"Fetching events for venue: \033[1;34m{venue_name}\033[0m (ID: {venue_id})")
    
    # Fetch events for the current venue
    venue_events = get_future_concerts_for_venue(venue_id)
    
    # Check if the fetched DataFrame is empty
    if venue_events.empty:
        print(f"No valid music events found for venue: {venue_name}. Skipping...")
        continue
    
    # Add columns for the venue ID and venue name to keep track of events by venue
    venue_events["Venue ID"] = venue_id
    venue_events["Venue Name"] = venue_name
    
    # Append the events for this venue to the list
    all_venues_events.append(venue_events)

# Concatenate all venue DataFrames into a single DataFrame
if all_venues_events:
    all_venues_events_df = pd.concat(all_venues_events, ignore_index=True)
    print("Successfully fetched events for all venues.")
    
    # Save the combined DataFrame to a CSV file
    output_file = "all_venues_events.csv"
    all_venues_events_df.to_csv(output_file, index=False)
    print(f"Data successfully saved to {output_file}")
else:
    print("No valid music events found for any venue.")

Fetching events for venue: SoFi Stadium (ID: KovZ917ACh0)
Fetching events for venue: Kia Forum (ID: KovZpZAEkn6A)
Fetching events for venue: Honda Center (ID: KovZpakTme)
Fetching events for venue: Hollywood Pantages Theatre (ID: KovZpZAEeFaA)
No more events found.
No valid music events found for venue: Hollywood Pantages Theatre. Skipping...
Fetching events for venue: Crypto.com Arena (ID: KovZpZAEdntA)
Fetching events for venue: Intuit Dome (ID: KovZ917Acq0)
Fetching events for venue: North Island Credit Union Amphitheatre (ID: KovZpa2WZe)
No valid music events found for venue: North Island Credit Union Amphitheatre. Skipping...
Fetching events for venue: The Wiltern (ID: KovZpZAEAl6A)
Fetching events for venue: Rose Bowl (ID: KovZpZAEA66A)
Fetching events for venue: House of Blues Anaheim (ID: KovZpZAEA67A)
Fetching events for venue: Hollywood Bowl (ID: KovZpZAFFvJA)
Fetching events for venue: YouTube Theater (ID: KovZ917AJue)
Error calculating years active for Yuridia: ('Connection

In [7]:
# Display the combined DataFrame
all_venues_events_df.columns
all_venues_events_df.to_csv(file_name, index=False)

Index(['Event ID', 'Event Name', 'Venue Name', 'Main Artist',
       'Supporting Artists', 'Supporting Artist Popularity', 'Total Tickets',
       'Date of Show', 'Min Ticket Price', 'Max Ticket Price',
       'Spotify Artist Name', 'Popularity Score', 'Followers', 'Genre',
       'Years Active', 'Public Onsale Date', 'Days to Event',
       'Days Since Onsale', 'Day of Week', 'Is Weekend', 'Venue ID'],
      dtype='object')

## Integrating Real-Time Reddit Mentions into Event Dataset

Fetches the count of Reddit mentions for artists based on today's posts using Reddit's OAuth2 API. The data is integrated into the event dataset to include a new feature, "Reddit Mentions Today," for each event. This involves:

- Authenticating with Reddit API using client credentials.
- Querying Reddit for posts mentioning the artist.
- Filtering posts by today's date to calculate mentions.
- Iterating through the event DataFrame to fetch mentions for each artist.
- Adding the calculated mentions as a new column in the dataset.

In [9]:
# Load environment variables
load_dotenv("Data_api.env")
REDDIT_CLIENT_ID = os.getenv("REDDIT_CLIENT_ID")
REDDIT_SECRET = os.getenv("REDDIT_SECRET")

def get_reddit_mentions_today(artist_name):
    """
    Fetch the count of Reddit mentions for a given artist name for today only.
    """
    # Reddit OAuth2 Authentication
    auth = requests.auth.HTTPBasicAuth(REDDIT_CLIENT_ID, REDDIT_SECRET)
    headers = {"User-Agent": "ArtistMentionTracker/0.1"}
    data = {"grant_type": "client_credentials"}

    try:
        # Request access token
        token_response = requests.post(
            "https://www.reddit.com/api/v1/access_token",
            auth=auth,
            data=data,
            headers=headers,
        )
        token_response.raise_for_status()
        token = token_response.json()["access_token"]
        headers["Authorization"] = f"bearer {token}"

        # Fetch Reddit mentions
        search_url = "https://oauth.reddit.com/search"
        params = {
            "q": artist_name,
            "limit": 100,  # Fetch up to 100 posts mentioning the artist
            "sort": "new",  # Sort by newest posts
        }
        response = requests.get(search_url, headers=headers, params=params)
        response.raise_for_status()
        posts = response.json()["data"]["children"]

        # Filter posts by today's date
        today = datetime.now(timezone.utc).date()
        mentions_today = sum(
            1 for post in posts if datetime.fromtimestamp(post["data"]["created_utc"], tz=timezone.utc).date() == today
        )
        return mentions_today

    except Exception as e:
        print(f"Error fetching Reddit mentions for {artist_name}: {e}")
        return 0

# Function to add Reddit mentions for the current day to a DataFrame
def add_reddit_mentions_today_to_dataset(df):
    """
    Add a 'Reddit Mentions Today' column to the existing DataFrame based on the 'Main Artist' column.
    """
    reddit_mentions_today = []

    # Loop through each artist and fetch mentions for today
    for artist in df['Event Name']:
        if artist != "N/A":  # Skip artists marked as N/A
            mentions = get_reddit_mentions_today(artist)
        else:
            mentions = 0  # Default to 0 for missing artists
        reddit_mentions_today.append(mentions)

    # Add the new column to the DataFrame
    df['Reddit Mentions Today'] = reddit_mentions_today
    return df

# Add Reddit mentions to the all_venues_events_df DataFrame
all_venues_events_df = add_reddit_mentions_today_to_dataset(all_venues_events_df)

# Display the updated DataFrame
all_venues_events_df

,Event ID,Event Name,Venue Name,Main Artist,Supporting Artists,Supporting Artist Popularity,Total Tickets,Date of Show,Min Ticket Price,Max Ticket Price,...,Followers,Genre,Years Active,Public Onsale Date,Days to Event,Days Since Onsale,Day of Week,Is Weekend,Venue ID,Reddit Mentions Today
0,vvG1IZbpD1R0Bh,Stray Kids World Tour [Dominate Los Angeles],SoFi Stadium,Stray Kids,N/A,0.0,4,2025-05-31,59.5,229.5,...,17420974,k-pop,6,2024-11-22T23:00:00Z,185,3,Saturday,1,KovZ917ACh0,34
1,vvG1IZbpxdMsDv,Julión Alvarez - 4218,SoFi Stadium,Julion Alvarez,N/A,0.0,8,2025-04-19,108.0,578.0,...,7195642,banda,17,2024-11-15T20:00:00Z,143,10,Saturday,1,KovZ917ACh0,0
2,vvG1IZbpD2BPda,Stray Kids World Tour [Dominate Los Angeles],SoFi Stadium,Stray Kids,N/A,0.0,4,2025-06-01,59.5,229.5,...,17420974,k-pop,6,2024-11-22T23:00:00Z,186,3,Sunday,1,KovZ917ACh0,34
3,vvG1IZb45xOfGf,Shakira - Las Mujeres Ya No Lloran - World Tour,SoFi Stadium,Shakira,N/A,0.0,6,2025-06-20,65.0,255.0,...,35526709,colombian pop,29,2024-10-25T19:00:00Z,205,31,Friday,0,KovZ917ACh0,0
4,vvG1IZbpSSPV1Z,Julión Alvarez - 4218,SoFi Stadium,Julion Alvarez,N/A,0.0,8,2025-04-18,128.0,598.0,...,7195642,banda,17,2024-11-14T21:15:00Z,142,11,Friday,0,KovZ917ACh0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910,vvG10ZbuEATSFI,Josey Scott The Original Voice Of Saliva,The Parish at House of Blues Anaheim,Josey Scott,"Red Devil Vortex, Blacklist Union",23.0,8,2024-12-05,25.0,25.0,...,2267,N/A,N/A,2024-06-28T17:00:00Z,8,150,Thursday,0,KovZ917AGEf,45
911,vv1AaZk7pGkdblF_z,Bo Napoleon,The Parish at House of Blues Anaheim,Bo Napoleon,The March,32.0,8,2024-12-28,20.0,20.0,...,24460,jawaiian,N/A,2024-08-30T17:00:00Z,31,87,Saturday,1,KovZ917AGEf,0
912,vv170ZbpGkDd2NDQ,The Wellermen,The Parish at House of Blues Anaheim,The Wellermen,N/A,0.0,8,2025-03-21,30.0,30.0,...,115749,shanty,1,2024-11-22T15:00:00Z,114,3,Friday,0,KovZ917AGEf,0
913,vv1Fe8vbGv19Z75G7A,Mikel Erentxun - El Último Vuelo Del Hombre Ba...,The Parish at House of Blues Anaheim,Mikel Erentxun,N/A,0.0,8,2025-03-17,27.5,27.5,...,174424,cantautor,32,2020-01-24T18:00:00Z,110,1767,Monday,0,KovZ917AGEf,4


In [11]:
all_venues_events_df.to_csv("all_venues_events.csv", index=False)